## Final Project Part 2
### Names: 
#### Mengyue Huang, Mingrui Xu

In [1]:
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
import bqplot
import altair as alt

In [2]:
df = pd.read_csv('https://query.data.world/s/b2okb3eqtyqkc4s7wbyblu3zr6tzbl')
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


The size of the dataset is 41KB, with 395 rows and 33 columns, thus it's within GitHub upload limit.

In [3]:
table = pd.pivot_table(df, values='Walc', index='Mjob', columns='Fjob', aggfunc = np.max, fill_value=0)
table

Fjob,at_home,health,other,services,teacher
Mjob,,,,,
at_home,3,1,5,5,1
health,0,4,5,4,3
other,3,2,5,5,2
services,2,4,5,5,3
teacher,2,5,5,4,5


In [4]:
myLabel = ipywidgets.Label()
def on_selected(change):
    i,j = change['owner'].selected[0]
    v = table.iat[i, j]
    myLabel.value = 'weekend alcohol consumption = ' +str(v)
    License_Type = table.index[i]
    license_Status = table.columns[j]
    

# 2. scales
col_sc = bqplot.ColorScale(scheme='Greens')
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# 3. axis
col_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
x_ax = bqplot.Axis(scale=x_sc, label='Father job')
y_ax = bqplot.Axis(scale=y_sc, label='Mother job', orientation='vertical')


# 4. marks
heat_map = bqplot.GridHeatMap(color = table.values,
                              column = table.columns.values.astype('str'), 
                              row = table.index.values.astype('str'), 
                              scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                              interactions = {'click':'select'},
                              anchor_style = {'fill':'blue'},
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})


fig = bqplot.Figure(marks=[heat_map], axes=[col_ax, x_ax, y_ax])
fig

heat_map.observe(on_selected, 'selected')
myDashboard = ipywidgets.VBox([myLabel, fig])
myDashboard

In [5]:
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()
ax_xcl = bqplot.Axis(label='weekend alcohol consumption', scale=x_scl)
ax_ycl = bqplot.Axis(label='mean health level', scale=y_scl, orientation='vertical', side='left')

i,j = 2,3
mask = (df['Mjob'] == table.index[i]) & \
        (df['Fjob'] == table.columns[j])
subset = df[mask]

grouped = subset.groupby("health")["Walc"].mean()

w_alc = grouped.index
mean_health_level = grouped.values

bar = bqplot.Bars(x = w_alc, y = mean_health_level,
                    scales = {'x': x_scl, 'y': y_scl})
fig_bar = bqplot.Figure(marks = [bar], axes = [ax_ycl, ax_xcl])
fig_bar

Figure(axes=[Axis(label='mean health level', orientation='vertical', scale=LinearScale(), side='left'), Axis(l…

In [6]:
def get_data_value(change):
    i,j = change['owner'].selected[0]
    mask = (df['Mjob'] == table.index.values[i]) & \
            (df['Fjob'] == table.columns.values[j])
    subset = df[mask]
    grouped = subset.groupby("health")["Walc"].mean()
    years = grouped.index
    values = grouped.values
    bar.x = years
    bar.y = values

heat_map.observe(get_data_value, 'selected')

fig.layout.min_width='500px'
fig_bar.layout.min_width = '500px'
figures = ipywidgets.HBox([fig, fig_bar])

myDashboard = ipywidgets.VBox([myLabel, figures])
myDashboard

The axes of the heatmap are the selected student's parents' jobs, which can be divided into "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other". Therefore, we could use the heatmap to check what types of parents' jobs will have children with very high alcohol consumption rate (with 0 means no and 5 means heavy alcohol consumption). The bar plot displays the students' alcohol consumption during weekends and the mean level of their health (with 0 means very bad and 5 means very good). When using this dashboard, the user can easily link the student's alcohol consumption with their health situation and their parents' job types. 

Contextual dataset: https://data.world/data-society/student-alcohol-consumption/workspace/file?filename=student-por.csv

Our dataset contains the information about students' alcohol consumption based on their parents' jobs, gender, and study information. Since the data were obtained from a survey of a math courses, we think that the contextual dataset can be the ones form other courses of the same high school. Therefore, the URL above contains the data form a portuguese language course and has the same categories of information as the math course. Therefore, instead of only focusing on the social or family background of a student, we can also check if the course they are taking has connect to their alcohol consumption levels. 